# Projeto 3 - Fazendo um modelo de investimento com Python - Factor Investing no Ibovespa.


### Desafio:

* Construir um código que faça um backtesting dos últimos 6 anos, escolhendo as 8 melhores ações do índice ibovespa e utilizando como critério o fator momento 7 meses.

### Passo a passo da estratégia:

   **Passo 1** - Definir um universo investível.

   **Passo 2** - Escolher o fator que servirá como critério para criação dos rankings.
   
   **Passo 3** - Escolher o período de teste.
   
   **Passo 4** - Escolher o número de ações na carteira. 
   
   **Passo 5** - Definir o período de balanceamento. De quanto em quanto tempo a carteira muda?
   
### Passo a passo do código? 

   **Passo 1** - Ler a composição histórica do Ibovespa e os tickers que já passaram pelo índice.
   
   **Passo 2** - Puxar as cotações de todas as empresas que farão parte do backtest.
   
   **Passo 3** - Transformar o índice em data e ordenar a série de tempo. 

   **Passo 4** - Calcular a média dos retornos 7 meses e ajustar a tabela com o fator. 
   
   **Passo 5** - Classificar e retirar empresas que não participaram do Ibovespa no período de tempo selecionado.  
   
   **Passo 6** - Criar as carteiras de investimento em uma matriz de 0 ou 1. 
   
   **Passo 7** - Calcular o retorno mensal das empresas no período de backtest. 

   **Passo 8** - Cruzar a matriz de retorno mensal com a matriz das carteiras para chegar na rentabilidade do modelo.
   
   **Passo 9** - Puxar e calcular a rentabilidade do Ibovespa no período. 
   
   **Passo 10** - Calcular e visualizar as rentabilidades do modelo contra o Ibovespa. 

In [ ]:
import yfinance as yf
import pandas as pd
import quantstats as qs

# Passo 1 - Ler a composição histórica do Ibovespa e os tickers que já passaram pelo índice.

In [ ]:
comp_historica = pd.read_excel('composicao_ibov.xlsx')
tickers = pd.read_excel('composicao_ibov.xlsx', sheet_name = 'lista_acoes')


tickers

In [ ]:
comp_historica

# Passo 2 - Puxar as cotações de todas as empresas que farão parte do backtest. 

In [ ]:
dados_cotacoes = yf.download(tickers = tickers['tickers'].to_list(),
                            start = "2015-05-29", end = "2022-12-31")['Adj Close']

dados_cotacoes

# Passo 3 - Transformar o índice em data e ordenar a série de tempo. 

In [ ]:
dados_cotacoes.index = pd.to_datetime(dados_cotacoes.index)
dados_cotacoes = dados_cotacoes.sort_index()

dados_cotacoes


# Passo 4 - Calcular a média dos retornos nos últimos 7 meses e ajustar a tabela com o fator. 

In [ ]:
r7 = (dados_cotacoes.resample("M").last().pct_change().rolling(7).mean().
                    dropna(axis = 0, how = "all").drop('2022-12-31'))

r7

# Passo 5 - Classificar e retirar empresas que não participaram do Ibovespa no período de tempo selecionado. 

In [ ]:
for data in r7.index:
    for empresa in r7.columns:

        if empresa.replace(".SA", "") not in comp_historica.loc[:, data].to_list():

            r7.loc[data, empresa] = pd.NA
            
r7

# Passo 6 - Criar as carteiras de investimento em uma matriz de 0 ou 1. 

In [ ]:
carteiras = r7.rank(axis = 1, ascending = False)

for data in carteiras.index:
    for empresa in carteiras.columns:

        if carteiras.loc[data, empresa] < 9:

            carteiras.loc[data, empresa] = 1
            
        else:
            
            carteiras.loc[data, empresa] = 0
            
            
            
carteiras

# Passo 7 - Calcular o retorno mensal das empresas no período de backtest. 

In [ ]:
retorno_mensal = dados_cotacoes.resample("M").last().pct_change()
retorno_mensal = retorno_mensal.drop(retorno_mensal.index[:8], axis = 0)
carteiras.index = retorno_mensal.index

carteiras

In [ ]:
retorno_mensal

# Passo 8 - Cruzar a matriz de retorno mensal com a matriz das carteiras para chegar na rentabilidade do modelo.

In [ ]:
comp_historica = pd.read_excel('composicao_ibov.xlsx')
tickers = pd.read_excel('composicao_ibov.xlsx', sheet_name = 'lista_acoes')

dados_cotacoes = yf.download(tickers = tickers['tickers'].to_list(),
                            start = "2015-05-29", end = "2022-12-31")['Adj Close']

dados_cotacoes.index = pd.to_datetime(dados_cotacoes.index)
dados_cotacoes = dados_cotacoes.sort_index()

r7 = (dados_cotacoes.resample("M").last().pct_change().rolling(7).mean().
                    dropna(axis = 0, how = "all").drop('2022-12-31'))


for data in r7.index:
    for empresa in r7.columns:

        if empresa.replace(".SA", "") not in comp_historica.loc[:, data].to_list():

            r7.loc[data, empresa] = pd.NA

carteiras = r7.rank(axis = 1, ascending = False)

for data in carteiras.index:
    for empresa in carteiras.columns:

        if carteiras.loc[data, empresa] < 9:

            carteiras.loc[data, empresa] = 1
            
        else:
            
            carteiras.loc[data, empresa] = 0

            
retorno_mensal = dados_cotacoes.resample("M").last().pct_change()
retorno_mensal = retorno_mensal.drop(retorno_mensal.index[:8], axis = 0)
carteiras.index = retorno_mensal.index

retorno_modelo = (carteiras * retorno_mensal).sum(axis = 1)/8

retorno_modelo

In [ ]:
retorno_modelo = (carteiras * retorno_mensal).sum(axis = 1)/8

# Passo 8.2 - Visualizar a rentabilidade do modelo com o quantstats.

In [ ]:
qs.extend_pandas()

retorno_modelo.plot_monthly_heatmap()

# Passo 9 - Puxar e calcular a rentabilidade do Ibovespa no período.

In [ ]:
ibovespa = yf.download("^BVSP", start = "2015-12-30", end = "2022-12-31")['Adj Close']

retornos_ibovespa = ibovespa.resample("M").last().pct_change().dropna()

retornos_ibovespa

# Passo 10 -  Calcular e visualizar as rentabilidades do modelo contra o Ibovespa. 

In [ ]:
retorno_acum_modelo = (1 + retorno_modelo).cumprod() - 1 

retorno_acum_ibov = (1 + retornos_ibovespa).cumprod() - 1 

retorno_acum_modelo.plot_monthly_heatmap()

retorno_acum_ibov.plot_monthly_heatmap()

# Passo 11 -  Calcular e visualizar as rentabilidades do modelo contra o Ibovespa. 

In [ ]:
overperfom_modelo_menos_ibov = retorno_modelo - retornos_ibovespa

overperfom_modelo_menos_ibov.plot_monthly_heatmap()